In [1]:
"""Find sample input size for neural network trainings logged on comet.ml"""
# pylint: disable=import-error, redefined-outer-name

In [2]:
from __future__ import annotations

from pathlib import Path
from typing import Dict

import pandas as pd
from comet_ml.api import API

In [ ]:
api = API()

In [ ]:
def collect_input_sizes(api: API) -> Dict[str, int]:
    """Collect NN input sizes metadata from comet.ml."""
    input_sizes = {}
    for experiment in api.get("rabyj/epilap"):  # type: ignore
        for hparam_dict in experiment.get_parameters_summary():
            if hparam_dict["name"] == "input_size":
                input_size = int(hparam_dict["valueMax"])
                meta = experiment.get_metadata()
                input_sizes[meta["experimentKey"]] = input_size
    return input_sizes

In [ ]:
input_sizes = collect_input_sizes(api)

In [ ]:
input_sizes_df = pd.DataFrame.from_dict(
    data=input_sizes, orient="index", columns=["input_size"]
)
input_sizes_df.to_csv("input_sizes.csv")

Merge input size with comet-ML general log.

In [ ]:
global_log_df = pd.read_csv(
    Path.home() / "Downloads/temp/cometml_dfreeze2.1_logs.csv", index_col=False, header=0
)

joined_df = global_log_df.merge(
    input_sizes_df, left_on="Experiment key", right_index=True, how="left"
)

new_input_size_df = (
    joined_df[["Experiment key", "input_size"]].set_index("Experiment key").sort_index()
)
new_input_size_df.to_csv(Path.home() / "Downloads/temp/new_input_sizes.csv")